In [90]:
import os
import numpy as np
from scipy.io import loadmat
import h5py
from tqdm import tqdm
import cv2
from fuel.datasets.hdf5 import H5PYDataset

In [86]:
def preprocess(hwc_bgr_image, size):
    hwc_rgb_image = cv2.cvtColor(hwc_bgr_image, cv2.COLOR_BGR2RGB)
    resized = cv2.resize(hwc_rgb_image, (size))
    chw_image = np.transpose(resized, axes=(2, 0, 1))
    return chw_image

In [75]:
image_filepath = "data/cars_train"
label_filepath = "data/car_devkit/devkit/cars_train_annos.mat"
data_path = "data/"

In [76]:
jpg_filenames = (os.listdir(image_filepath))
jpg_filenames.sort()
num_examples = len(jpg_filenames)

In [77]:
cars_annos = loadmat(label_filepath)
annotations = cars_annos["annotations"].ravel()
annotations = sorted(annotations, key=lambda a: str(a[5][0]))

In [78]:
class_labels = []
for annotation in annotations:
    class_label = int(annotation[4])
    class_labels.append(class_label)

In [80]:
hdf5_filename = "cars196.hdf5"
hdf5_filepath = os.path.join(data_path, hdf5_filename)
hdf5 = h5py.File(hdf5_filepath, mode="w")

In [81]:
# store images
image_size = (256, 256)
array_shape = (num_examples, 3) + image_size
ds_images = hdf5.create_dataset("images", array_shape, dtype=np.uint8)
ds_images.dims[0].label = "batch"
ds_images.dims[1].label = "channel"
ds_images.dims[2].label = "height"
ds_images.dims[3].label = "width"

In [87]:
# write images to the disk
for i, filename in tqdm(enumerate(jpg_filenames), total=num_examples, desc=hdf5_filepath):
    raw_image = cv2.imread(os.path.join(image_filepath, filename), cv2.IMREAD_COLOR)  # BGR image
    image = preprocess(raw_image, image_size)
    ds_images[i] = image

data/cars196.hdf5: 100%|██████████| 8144/8144 [02:15<00:00, 59.92it/s] 


In [88]:
# store the targets (class labels)
targets = np.array(class_labels, np.int32).reshape(num_examples, 1)
ds_targets = hdf5.create_dataset("targets", data=targets)
ds_targets.dims[0].label = "batch"
ds_targets.dims[1].label = "class_labels"

In [91]:
# specify the splits (labels 1~98 for train, 99~196 for test)
test_head = class_labels.index(99)
split_train, split_test = (0, test_head), (test_head, num_examples)
split_dict = dict(train=dict(images=split_train, targets=split_train),test=dict(images=split_test, targets=split_test))
hdf5.attrs["split"] = H5PYDataset.create_split_array(split_dict)
hdf5.flush()
hdf5.close()  

/home/mayank/.virtualenvs/test/lib/python3.6/site-packages/fuel/datasets/hdf5.py:270: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  ('comment', 'a', comment_len)]))
